In [27]:
import numpy as np
import pandas as pd
import json
import warnings
from os.path import join, expanduser
import pickle

In [28]:
data_dir = expanduser('~/Documents/temp/data/snowschool/station/')
meas = pd.read_csv(join(data_dir,'winter_20_21_clean.csv'), index_col = 0, parse_dates= ['Unnamed: 0'])
units = meas.iloc[0]
meas = meas.iloc[1:]
meas.index = pd.to_datetime(meas.index)
meas = meas.tz_convert('US/Mountain')
air_temp_cols = []
for col in meas.columns:
    if 'air_temp' in col:
        air_temp_cols.append(col)
    try:
        meas[col] = meas[col].astype(float)
    except Exception as e:
        print(e)
meas.surface_temp_1 = meas.surface_temp_1.interpolate()

could not convert string to float: 'NNW'


In [29]:
with open('/Users/zachkeskinen/Documents/temp/data/snowschool/station/df.pkl', 'wb') as f:
    pickle.dump(meas, f)

In [31]:
fp = '/Users/zachkeskinen/Documents/temp/modeling/output/v20220210-aspectright_forcedepthmatch/SnowSchool 1.timeline.json'
# Opening JSON file
f = open(fp,)
# returns JSON object as
# a dictionary
data = json.load(f)
# Closing file
f.close()

In [32]:
model = pd.DataFrame()
for profile in data['profiles']:
    model = model.append(profile, ignore_index= True)

/var/folders/0n/swn__wf94qd3zdch1tvp087r0000gn/T/ipykernel_86210/3352014247.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(profile, ignore_index= True)
/var/folders/0n/swn__wf94qd3zdch1tvp087r0000gn/T/ipykernel_86210/3352014247.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(profile, ignore_index= True)
/var/folders/0n/swn__wf94qd3zdch1tvp087r0000gn/T/ipykernel_86210/3352014247.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  model = model.append(profile, ignore_index= True)
/var/folders/0n/swn__wf94qd3zdch1tvp087r0000gn/T/ipykernel_86210/3352014247.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.

In [33]:
import pickle
with open('/Users/zachkeskinen/Documents/temp/data/snowschool/modeled/full_model.pkl', 'wb') as f:
    pickle.dump(model, f)

In [34]:
model_depths = pd.DataFrame(index = meas.index, columns = ['hs'])
for i, r in model.iterrows():
    model_depths.loc[pd.to_datetime(r.date), 'hs']= r.comments['elements'][0]['layers'][-1]['top']/100
model_depths.hs = model_depths.hs.astype('f8')

/var/folders/0n/swn__wf94qd3zdch1tvp087r0000gn/T/ipykernel_86210/2493658171.py:3: FutureWarning: Setitem-like behavior with mismatched timezones is deprecated and will change in a future version. Instead of raising (or for Index, Series, and DataFrame methods, coercing to object dtype), the value being set (or passed as a fill_value, or inserted) will be cast to the existing DatetimeArray/DatetimeIndex/Series/DataFrame column's timezone. To retain the old behavior, explicitly cast to object dtype before the operation.
  model_depths.loc[pd.to_datetime(r.date), 'hs']= r.comments['elements'][0]['layers'][-1]['top']/100


In [35]:
model_depths.to_csv('/Users/zachkeskinen/Documents/temp/data/snowschool/modeled/depths.csv')

In [36]:
from tqdm import tqdm
heights = np.linspace(0,1.4,29)
extracted_temps = pd.DataFrame(columns = heights, index = meas.index)
for i, r in tqdm(model.iterrows()):
    if type(r.temperature) is not float:
        for layer in r.temperature['elements'][0]['layers']:
            layer_heights = heights[(heights <= layer['top']/100) & (heights >= layer['bottom']/100)]
            if layer_heights.size > 0:
                for h in layer_heights:
                    extracted_temps.loc[pd.to_datetime(r.date), h] = layer['value']
mod_temp_arr = extracted_temps.values.astype('f8')

4297it [00:20, 210.69it/s]


In [39]:
constant_dir = expanduser('~/Documents/temp/src/functions/constants/')

with open(join(constant_dir, 'sensor_heights.pkl'), 'rb') as f:
    sensor_heights = pickle.load(f)
inv_map = {v: k for k, v in sensor_heights.items()}

In [48]:
model_csv = pd.DataFrame(mod_temp_arr, columns = [inv_map[round(h,2)] for h in heights], index = meas.index)

In [50]:
model = model_csv
model.loc[:, 'snow_depth_1'] = meas.snow_depth_1.interpolate()
model.loc[:, 'air_temp_1'] =  meas.air_temp_1

In [51]:
model.to_csv('/Users/zachkeskinen/Documents/temp/data/snowschool/modeled/temps.csv')

In [52]:
with open('/Users/zachkeskinen/Documents/temp/data/snowschool/modeled/temps.pkl', 'wb') as f:
    pickle.dump(model, f)